In [1]:
## GLADE+ galaxy catalogue:
## Column 1 : GLADEno;          GLADE+ catalog number 
## Column 3 : GWGCname;         Name in the GWGC catalog
## Column 9 : RA                Right ascension in degrees
## Column 10: Dec               Declination in degrees
## Column 14: B_Abs             Absolute B magnitude
## Column 28  z                 Redshift in the heliocentric frame
## Column 32  z_err             Measurement error of heliocentric redshift
## Column 33: d_L               Luminosity distance in Mpc units
## Column 34: d_Lerr            Error of luminosity distance in Mpc units
## Column 36: Mst               Stellar mass in 10^10 M_Sun units
## Column 37: Mst_err           Absolute error of stellar mass in 10^10 M_Sun units
## Column 39: Merger_rate       Base-10 logarithm of estimated BNS merger rate in the galaxy in Gyr^-1 units
## Column 40: Merger_rate_error Absolute error of estimated BNS merger rate in the galaxy
## For GLADE 2.4:
## Column 1 : GLADEno;          GLADE+ catalog number 
## Column 2 : GWGCname;         Name in the GWGC catalog
## Column 7 : RA                Right ascension in degrees
## Column 8: Dec                Declination in degrees
## Column 14: B_Abs             Absolute B magnitude
## Column 9: d_L                Luminosity distance in Mpc units
## Column 10: d_Lerr               Error of luminosity distance in Mpc units
## Column 11  z                 Redshift in the heliocentric frame

In [2]:
## import packages, need healpy to read .fits files
import numpy as np
import pandas as pd
import healpy as hp
import matplotlib.pyplot as plt
import math

In [3]:
## the GW events we are going to use : for simplicity, for BNS events we only use high-spin analysis results
localisation_data_name = ['../../../GW170817/localisation/skymap.fits']
GW_post_name = ['../../../GW170817/posterior/high_spin_PhenomPNRT_posterior_samples.dat']
prior_name = ['../../../GW170817/prior/high_spin_PhenomPNRT_posterior_samples.dat']
GC_name = '../../../GC/GLADE_2.4.txt'
# set up the bin_num for later luminosity analysis
bin_num = 80
if_savetxt = 1 # do you want to save the txt files of the overall skymap, dL, ranking, and evidence
if_plot = 1 # do you want to plot the posterior skymap, the distribution of dL, list of the ranking
tot_incomp = 0.1 # the total incompleteness of the GC we use

In [4]:
## define a function that loads the localisation posterior from GW
def read_fits(filename):
    skymap,header = hp.read_map(filename, h=True)
    # Get the number of pixels in the map
    nside = hp.get_nside(skymap)
    npix = hp.nside2npix(nside)

    # Create an empty array to hold the coordinates
    coords = np.empty((2,npix))

    # Loop over all the pixels in the map and get the coordinates
    for i in range(npix):
        theta, phi = hp.pix2ang(nside, i, nest=False, lonlat=True)
        coords[0,i] = theta  # Right ascension
        coords[1,i] = 90.0 - phi  # Declination
    RA = coords[0,:]
    Dec = coords[1,:]
    return RA, Dec, skymap,nside,npix


In [5]:
## define a function that loads the galaxy catalogue GC: GLADE+
def read_gc(GC_name, minRA, maxRA, minDec, maxDec, mindL, maxdL):
    GC = np.loadtxt(GC_name, dtype = 'str');
    ## count the number of galaxies we need
    GLADEno = []; GWGCname = []; RA = []; Dec = []; B_Abs = []; d_Lerr = [];z = [];z_err = [];
    d_L = []; Mst = []; Mst_err = []; Merger_rate = []; Merger_rate_error = [];
    for i in range(len(GC[:,1])):
        ## consider the error bar of the luminosity distance of every galaxy
        d_temp = float(GC[i][32]); d_temp_err = float(GC[i][33]);
        ra_temp = float(GC[i][8]); dec_temp = float(GC[i][9]);
        if  (ra_temp<maxRA&ra_temp<minRA)&(dec_temp<maxDec&dec_temp>minDec)&((d_temp-d_temp_err)<maxdL&(d_temp+d_temp_err)>mindL): 
            GLADEno.append(int(GC[i][0])); GWGCname.append(GC[i][2]); 
            RA.append(float(GC[i][8]));Dec.append(float(GC[i][9])); 
            B_Abs.append(float(GC[i][13])); d_L.append(float(GC[i][32])); 
            Mst.append(float(GC[i][35])); Mst_err.append(float(GC[i][36])); 
            Merger_rate.append(float(GC[i][38])); Merger_rate_error.append(float(GC[i][39]));
            z.append(float(GC[i][27]));z_err.append(float(GC[i][31])); 
    del GC
    return GLADEno,GWGCname,RA,Dec,B_Abs,d_L,z,z_err,Mst,Mst_err,Merger_rate,Merger_rate_error

## define a function that loads the galaxy catalogue GC: GLADE 2.4
def read_gc_2(GC_name, minRA, maxRA, minDec, maxDec, mindL, maxdL):
    GC = np.loadtxt(GC_name, dtype = 'str');
    ## count the number of galaxies we need
    GLADEno = []; GWGCname = []; RA = []; Dec = []; B_Abs = []; d_Lerr = [];z = [];
    d_L = []; Mst = []; Mst_err = []; Merger_rate = []; Merger_rate_error = [];
    for i in range(len(GC[:,1])):
        d_temp = float(GC[i][8]); d_temp_err = float(GC[i][9]);
        ra_temp = float(GC[i][6]); dec_temp = float(GC[i][7]);
        if  (ra_temp<maxRA&ra_temp<minRA)&(dec_temp<maxDec&dec_temp>minDec)&((d_temp-d_temp_err)<maxdL&(d_temp+d_temp_err)>mindL): 
            GLADEno.append(int(GC[i][0])); GWGCname.append(GC[i][1]); 
            RA.append(float(GC[i][6]));Dec.append(float(GC[i][7])); 
            B_Abs.append(float(GC[i][13])); d_L.append(float(GC[i][8])); 
            z.append(float(GC[i][10]));
    del GC
    return GLADEno,GWGCname,RA,Dec,B_Abs,d_L,z

In [6]:
## define a function that calculates the Multi-Messenger Prior Func (MMPF)
## this function takes the parameters from GW skymaps to......eh...draw a skymap
## We assume simple incompleteness of the GC to the range of the GW detectors in this function
def mmpf(GW_RA,GW_Dec,GW_dL_range,CG_RA,CG_Dec,GC_dL,B_Abs,tot_incomp):
    # for the incompleteness of GC, we assume 10% overall incompleteness here just for an example
    # we assume incomplenetess is isotropic. hence, it will not affect the skymap except reduce the probability of galaxies in the GC
    
    N = len(CG_RA) # we have N galaxies considered here
    MMPF_GC = np.zeros(len(GW_RA))
    MMPF_incomp = np.ones(len(GW_RA)) # set up the MMPF
    incomp = (tot_incomp/len(MMPF_incomp)) # assume the incompleteness is uniform across the sky
    MMPF_incomp *= incomp
    GC_onskymap = [] # set up the position of each galaxy on the skymap
    ## we want to find the step of GW_Dec
    ## because the exact number of GW_Dec and CG_Dec might be different, so we treat it like bins. 
    ## since the first few terms in GW_Dec is the same, we need to first find the second different term(sdt), index:sdt+1
    sdt = 0
    while CG_Dec[sdt]==CG_Dec[sdt+1]:
        sdt+=1
    err = abs(CG_Dec[0]-CG_Dec[sdt+1])
    
    # for every galaxy, we find the pixel of it in the GW skymap and attribute it a weight
    for i in range(N):
        # we compare declination first due to the data structure of the skymap(same dec, different ra)
        dec_index = np.where(abs(GW_Dec-CG_Dec[i])<err/2)
        # then the same bin size step for ra, just like the above dec case
        temp_RA = GW_RA[dec_index]
        err_ra = abs(temp_RA[1]-temp_RA[2])
        final_index = np.where(abs(GW_RA[dec_index]-CG_RA[i])<err_ra/2)
        # the final_index represents the location of the galaxy in the skymap
        GC_onskymap.append(final_index)
        weighting_norm = sum(B_Abs)/(1-tot_incomp)
        weighting = B_Abs[i]/weighting_norm
        MMPF_GC[final_index] = weighting
        
    # also need to generate our prior for dL
    # for GC terms
    GCPR_dL = np.zeros(len(GW_dL_range)-1);
    GC_dL_ind = np.zeros(len(GC_dL)) # mark where the dL of galaxes lies in the historam
    for i in range(len(GC_dL)):
        GC_dL[i] = galaxy_dis
        for j in range(len(GW_dL_range)-1):
            if (galaxy_dis>GW_dL_range[j])&(galaxy_dis<GW_dL_range[j+1]):
                GC_dL_ind[i] = j
                GCPR_dL[j] = weighting

    # for the incompleteness term
    # currently, we will just assume that the probability distribution of dL of a unmarked galaxy ouside the GC increases with dL^2
    # it will be replaced by V(z) in the future
    incomp_dL = np.ones(len(GW_dL_range)-1);
    for i in range(len(GW_dL_range)-1):
        step_len = (GW_dL_range[i] + GW_dL_range[i+1])/2
        incomp_dL[i] = step_len*step_len
    incomp_dL = incomp_dL*tot_incomp/sum(incomp_dL)

    
    return MMPF_GC,GC_onskymap,MMPF_incomp,GCPR_dL,incomp_dL,GC_dL_ind
    # skymaps and dL are independent for GW posterior and prior, but not independent for MMPC
    # NOTE that the MMPC_GC and CGPR_dL here are all marginalised, because they are both weighed
    # if you multiply them directly, you weill be weighing a galaxy twice

In [ ]:
## main part of the code
## set up the final output: posteriors of skymaps, B_Abs and dL, etc
post_skymap = [] 
post_dL = [] 
post_B_Abs = [] 
ranking= [] 
evidence_Mb = [] # evidence of taking M_b into account
## loop for every event we have
for idata in range(len(GW_post_name)):
    print('Now reading posterior data: '+GW_post_name[idata])
    # read the GW posterior data
    sample_data = GW_post_name[idata]
    localisation_data = localisation_data_name[idata]
    # for txt/dat GW posterior files
    GW_post_raw = np.loadtxt(sample_data, dtype = 'str')
    GW_post_raw = np.delete(GW_post_raw, 0, axis=0) # the first row is a header
    GW_post = GW_post_raw.astype(float) 
    GW_dL = np.transpose(GW_post[:,1]) # luminosity distance
    GW_M = np.transpose(np.hstack((GW_post[:,2],GW_post[:,3])))# redshifted mass 
    mindL = min(GW_dL); maxdL = max(GW_dL); # the range of the luminosity distance
    
    print('Now reading localisation data...')
    # for GW localisation fits files, we read the posterior probability distribution of RA and Dec, 
    # and post_loc represents the posterior localisation results (the sky map)
    GW_RA, GW_Dec, GW_skymap,nside,npix = read_fits(localisation_data)
    nz_positions = np.nonzero(GW_skymap) # find the non-zero positions in the skymap
    GW_RA_nz = GW_RA[nz_positions];
    GW_Dec_nz = GW_Dec[nz_positions]; 
    GW_skymap_nz = GW_skymap[nz_positions]; 
    maxRA = max(GW_RA[nz_positions]);minRA = min(GW_RA[nz_positions]) # rough range of ra and dec
    maxDec = max(GW_Dec[nz_positions]);minDec = min(GW_Dec[nz_positions])
    # we put the GW_dL data in bins to analyze
    GW_dL_range = np.linspace(math.floor(min(GW_dL)), math.ceil(max(GW_dL)), bin_num)
    GW_dL_hist, _ = np.histogram(GW_dL, bins=GW_dL_range)
    
    
    print('Now reading GC: GLADE 2.4...')
    # load the galaxy catalogue GC
    GLADEno,GWGCname,GC_RA,GC_Dec,B_Abs,GC_dL,z = read_gc_2(GC_name, minRA, maxRA, minDec, maxDec, mindL, maxdL)
    
    print('Computing MMPF...')
    # get the MMPF
    MMPF_GC,GC_onskymap,MMPF_incomp,GCPR_dL,incomp_dL,GC_dL_ind = mmpf(GW_RA_nz,GW_Dec_nz,GW_dL_range,GC_RA,GC_Dec,GC_dL,B_Abs,tot_incomp)
    
    
    
    # load the priors from LAL_inference package (by John)
    # for LAL_inference package, the prior for ra and dec is uniform acoross the sky (isotropy), so this will just be normalised
    # and the probability distribution of luminosity distance increases with dL^2 from 1 mpc to max designed range
    # the mass prior, the component mass is uniform distribution from 1 solar mass to 30, total mass less than 35
    PR_dL = np.ones(len(GW_dL_range));
    for i in range(len(GW_dL_range)):
        PR_dL[i] = GW_dL_range[i]*GW_dL_range[i]
        
        
    print('Now calculating final posterior...')    
    ## now compute the final posterior
    # marginalised ra and dec(sky location)
    post_skymap_idata = np.array(len(GW_skymap))
    post_skymap_idata_nz = (MMPF_GC + MMPF_incomp) * GW_skymap_nz
    post_skymap_idata_nz = post_skymap_idata_nz / np.sum(post_skymap_idata_nz) # normalisation
    j = 0
    for i in range(len(GW_skymap)):
        if GW_skymap[i]!=0:
            post_skymap_idata[i] = post_skymap_idata_nz[j]
            j += 1
    
    post_skymap.append(post_skymap_idata)
    if (ifplot):
        plt.figure((4*idata-3))
        hp.mollview(map=GW_skymap,fig=(4*idata-3),title='The skymap of GW localisation alone')
        plt.savefig(str(idata)+'_gwskymap_'+'.eps', format='eps')
        plt.figure((4*idata-2))
        hp.mollview(map=post_skymap_idata,fig=(4*idata-2),title='The skymap of multimessenger localisation')
        plt.savefig(str(idata)+'_multiskymap_'+'.eps', format='eps')
        
    
    # marginalised dL
    post_dL_idata = GW_dL_hist*(incomp_dL+GCPR_dL)/PR_dL
    post_dL.append(post_dL_idata)
    if (ifplot):
        plt.figure((4*idata-1))
        # Plot the histograms
        plt.hist(GW_dL_hist, bins=binsize, alpha=0.5, label='GW')
        plt.hist(post_dL_idata, bins=binsize, alpha=0.5, label='Multimessenger')

        # Add labels and legend to the plot
        plt.xlabel('dL/Mpc')
        plt.ylabel('Frequency')
        plt.title('Probability distribution of luminosity distance')
        plt.legend()
        plt.savefig(str(idata)+'_dL_'+'.eps', format='eps')
        
    # ranking of galaxies
    ranking_idata = []
    # we can just reweigh the galaxies on post_skymap_idata using the GW luminosity distribution
    # for every galaxy
    for i in range(GC_onskymap):
        gala_pos = GC_onskymap[i] # the position on the non-zero skymap
        gala_name = GWGCname[i]
        gala_num = GLADEno[i]
        gala_dis = GC_dL[i]
        gala_B_Abs = B_Abs[i]
        distance_factor = GW_dL_hist[GC_dL_ind[i]]/sum(GW_dL_hist)
        gala_probability = post_skymap_idata_nz[gala_pos]*distance_fac
        
        ranking = [gala_num,gala_name,gala_B_Abs,gala_probability]
        ranking_idata.append(ranking)
    
    
    ranking.append(ranking_idata)
    if (ifplot):
        print('The most possible hosting galaxies for this GW event are :')
        print(ranking)
        
    # B_Abs
    # In ideal case, we should add the distribution of B-band luminosity to the incompleteness term
    #post_B_Abs_idata = 
    #post_B_Abs.append(post_B_Abs_idata)
    if (ifplot):
        plt.figure((4*idata))
        plt.plot(ranking_idata[:,2],ranking_idata[:,3])
        plt.xlabel('B-band absolute luminosity')
        plt.ylabel('Frequency')
        plt.title('B-band luminosity of GW source hosting galaxies')
        plt.legend()
        plt.savefig(str(idata)+'_B_Abs_'+'.eps', format='eps')
        
    
    # overall evidence
    evidence_Mb_idata = sum(ranking_idata[3])*(1+tot_incomp/(1-tot_incomp))
    evidence_Mb.append(evidence_Mb_idata)
    

Now reading posterior data: ../../../GW170817/posterior/high_spin_PhenomPNRT_posterior_samples.dat
Now reading localisation data...
Now reading GC: GLADE 2.4...


In [ ]:
if (if_savetxt):
    np.savetxt('skymap_'+'.txt', post_skymap)
    np.savetxt('dL_'+'.txt', post_dL)
    #np.savetxt('B_Abs_'+'.txt', post_B_Abs)
    np.savetxt('ranking_'+'.txt', ranking)
    np.savetxt('evidence_Mb_'+'.txt', evidence_Mb)